In [1]:
!pip install rouge datasets==1.4.1 transformers==4.4.2

  Using cached rouge-1.0.1-py3-none-any.whl (13 kB)
  Using cached datasets-1.4.1-py3-none-any.whl (186 kB)
  Using cached transformers-4.4.2-py3-none-any.whl (2.0 MB)
  Using cached tqdm-4.49.0-py2.py3-none-any.whl (69 kB)
  Using cached huggingface_hub-0.0.2-py3-none-any.whl (24 kB)
  Using cached xxhash-2.0.2-cp36-cp36m-manylinux2010_x86_64.whl (243 kB)
  Using cached tokenizers-0.10.3-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.2
    Uninstalling tqdm-4.62.2:
      Successfully uninstalled tqdm-4.62.2
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


## preprocess data

In [89]:
## data process 
import pandas as pd
from datasets import load_dataset

df_article_summary_full = pd.read_parquet('../../../data/article_summary.parquet', engine='pyarrow')
df_article_summary_full[['original_text','summary']].to_csv('./total.csv',index=False)



dataset = load_dataset('csv', data_files='./total.csv',split='train')
dataset =  dataset.rename_column("original_text", "article")
dataset =  dataset.rename_column("summary", "summarization")
#train test split
dataset = dataset.train_test_split(test_size=0.1)

Using custom data configuration default-71d62c7c4532cf3a


Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-71d62c7c4532cf3a/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'summarization'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['article', 'summarization'],
        num_rows: 1500
    })
})

In [41]:
import os

path = './demo_data/SUMMARY.hk01'
os.makedirs(path, exist_ok=True)
print("The new directory is created!")

#dataset['train'].to_json(os.path.join(path,'train.json'),orient = 'records')
#dataset['test'].to_json(os.path.join(path,'test.json'), orient = 'records')

# use csv file to test 
dataset['train'].to_csv(os.path.join(path,'train.csv'))
dataset['test'].to_csv(os.path.join(path,'test.csv'))
dataset['test'].to_csv(os.path.join(path,'dev.csv'))

The new directory is created!


4334137

## run train

In [110]:
#!python run_gen.py --model_path 'fnlp/cpt-base' --dataset adgen --data_dir demo_data
!python run_gen_v2.py --model_path 'output/hk01/6' --dataset hk01 --data_dir demo_data --epoch '1' --batch_size '4' 

Using custom data configuration default-327463c7b77836c9
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-327463c7b77836c9/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
dataset:  DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'article', 'summarization'],
        num_rows: 13500
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'article', 'summarization'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'article', 'summarization'],
        num_rows: 1500
    })
})
10/20/2021 04:21:14 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/20/2021 04:21:14 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='output/hk01/8', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=

## rouge score calculate

In [84]:
!python run_bleu_v2.py

Using custom data configuration default-8b72df5a2fc7b575
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-8b72df5a2fc7b575/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
100
100
0.19607938757312063


## 推理

In [88]:
import sys
sys.path.append('../')

from transformers import BertTokenizer
from modeling_cpt import CPTModel, CPTForConditionalGeneration

model = CPTForConditionalGeneration.from_pretrained("output/hk01/6")
tokenizer = BertTokenizer.from_pretrained('output/hk01/6')

#model_inputs = tokenizer(test_set[0]['article'], max_length=128, padding="max_length", truncation=True,return_tensors="pt")

In [108]:
%%time
def print_result(idx):
    input_text = dataset['test'][idx]['article']
    print("input: ",input_text)

    inputs = tokenizer(input_text, return_tensors="pt",max_length=512)

    #prompt_length = len(tokenizer.decode(inputs['input_ids'][0]))
    #outputs = model.generate(inputs['input_ids'], max_length=64, do_sample=True, top_p=0.95, top_k=60)
    outputs = model.generate(inputs['input_ids'], max_length=64, top_p=0.95)
    generated = tokenizer.decode(outputs[0])

    print("prediction result: ",generated)
    
    print ('label: ',dataset['test'][idx]['summarization'])

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 11 µs


In [109]:
print_result(0)

input:  美國知名基督教牧師葛培理博士（Dr. Billy Graham）周三（21日）逝世，眾人皆知這位以演說鼓舞千萬信徒的佈道家，也是慈愛的父親與「愛妻號」。他1943年與同學鍾路得（Ruth Bell Graham）結婚。這位讓他一見傾心的女子，原來在中國出生和成長，曾留學朝鮮。她一直心懷亞洲民眾，鼓勵並陪同丈夫先後在1988年和1992年到中國及朝鮮進行歷史性訪問。 葛培理和鍾路得由同學成為終身伴侶，一同經歷64個寒暑。（billygraham.org圖片） 鍾路得1920年6月10日出生於江蘇清江浦（今淮安市）。父親鍾愛華（L. Nelson Bell）任職外科醫生，跟母親Virginia McCue Bell都是美南長老會（Presbyterian Church in the United States）派到中國的醫療傳教士。鍾路得在五兄弟姊妹中排行第二。 她在1933年、即13歲時被送到朝鮮平壤讀中學。父母3年後休假，她隨雙親返回家鄉美國北卡羅來納（North Carolina）蒙特列特鎮（Montreat），在此完成中學課程。 鍾路得1937年秋季進入伊利諾伊州惠頓學院（Wheaton College）升學。同校攻讀人類學的葛培理憶起兩人邂逅情景：「我看見她在路上朝我的方向走來的時候，我的視線無法移開。她望向我，我們視線相接，而我肯定她是我想娶的女人。」 葛培理雖然因為工作經常不在家，鍾路得靠書信和講電話與丈夫維繫感情。（billygraham.org圖片） 兩人大學畢業後不久就決定相守終生。他們1943年8月13日成婚後也選擇在蒙特列特鎮組織家庭。夫妻倆育有二子三女。由於葛培理經常在外工作，教育子女的重擔落在她身上。 她不僅是葛培理的賢內助，更是他事業路上的夥伴。1950年《抉擇時刻》（Hour of Decision）開始廣播，節目名稱便是她的手筆。鍾路得在中國成長與朝鮮受教育的背景，讓她對亞洲人懷有特殊深厚的感情，葛培理先後在1988年和1992年對中國和朝鮮訪問，鍾路得功不可沒。 夫婦倆1996年雙雙獲頒美國國會金章獎（Congressional Gold Medal），以表揚他們在德育、種族平等、家庭、公益和宗教事務的貢獻。 葛培理曾大讚妻子鍾路得肩負照顧兒女重擔，令他可以工作時得心應手。（billygraham.org圖片） 兩人攜

In [98]:
print_result(1)

input:  比利時布魯塞爾時間7月21日上午5時31分，歐洲理事會主席米歇爾（Charles Michel）在Twitter上推出「搞定！」（Deal!）一語，為這場原訂在17至18日完事、最後卻連續開了近90個小時的歐盟預算峰會劃上句號。在歐盟27國領袖五日四夜幾乎日夜不眠、每晚開會到日出的努力下，2020至2027年總值1.074萬億歐元的預算有了着落，而且首次由歐盟共同大額舉債融資、總值7,500億歐元的疫後「復蘇基金」也最終順利過關。 跟普遍的領袖峰會不同，歐盟國家數目太多，國與國之間的政治爭議繁瑣，難以先由級別較低的官員或技術官僚敲定細節再由各國領袖從容簽名。因此，各國領袖徹夜開會的情況其實並不罕見。不過，這次預算峰會也算是「表現突出」：對上一次歐盟各國連續開會超過90小時的情況要數到2000年的法國尼斯（Nice）峰會——當年歐盟15國的峰會，就為其後將歐盟擴展至今天27國鋪設了至今也極具爭議的決策權力分配機制。 會議是「廿年最長」還是「史上最長」？ 對於這次預算峰會有沒有打破當年的紀錄，如今儼如「節儉五國」（Frugal Five）之首、一直在各種層面反對「復蘇基金」的荷蘭首相呂特（Mark Rutte），與「復蘇基金」的主要受助國意大利總理孔特（Giuseppe Conte）也有不同意見。 在絕大多數國家都支持復甦基金的情況下，荷蘭首相呂特成為了這次峰會的反派主角。（美聯社） 意大利總理孔特，在歐盟通過復甦基金的無償資助後，大概可揭穿自己的親歐派面目。（美聯社） 這次幾乎獨力使會議由兩天變成五天的呂特表示，他曾經請示米歇爾他應否在峰會的最後時刻多講幾句，讓會議能打破尼斯峰會的時間紀錄，不過最後孔特卻拿了發言權。然而，最終會議大約在上午5時45分完結，呂特就指這次會議「最終完結得早了一些」。與呂特在預算問題上爭持不下、日前曾痛批呂特只能在荷蘭國內「當幾天英雄」的孔特，卻指會議很可能打破了尼斯峰會的時間紀錄。 根據追蹤歐盟多年的一位記者計算，這次疫情後各國領袖首次親身出席的峰會從17日的上午10時，一直開到21日的上午5時45分，總共91小時45分；而尼斯峰會由當年12月7日的上午9時45分，一直開到11日的上午5時30分，也剛好總長91小時45分。因此，呂特與孔特兩人皆錯。（不過尼斯峰會的時間是否要包括當時其他正要加入歐盟的國家領袖的會議，卻是另一

In [99]:
print_result(2)

input:  西班牙大選結束，目前點票結果顯示首相桑切斯所屬的工人社會黨勝出，但無法取得過半數議席。另一方面極右呼聲黨取得更多議席成為第三大黨，選舉結果顯示政治僵局仍難以打破。 工人社會黨支持慶祝選舉取得勝利，但這次選舉結果與上次差不多，難以為執政黨取得突破。（路透社） ↓↓↓想看更多西班牙加泰羅尼亞示威照片，請點擊放大觀看： 大約500名示威者11月11日在拉宏格亞市（La Jonquera）的邊界一帶封堵連接加泰羅尼亞與法國的交通要道AP-7高速公路，目的是促使西班牙明白，坐下來對談是唯一的解決方法。（Reuters） 有示威者在高速公路旁架起睡床休息。（Reuters） 警方排成一條橫線。（Reuters） 警方與堵塞公路的示威者溝通。（Reuters） 這次是西班牙4年以來第四次議會選舉，也是今年第二次。根據目前點票結果顯示，首相桑切斯（Pedro Sanchez）所屬的工人社會黨勝出，得120席。但不僅無法取得國會過半數，還要較大選前少數席。 而極右的呼聲黨（Vox），則憑着加泰羅尼亞危機的呼聲，議席增加一倍至56席，成為這次選舉中的另一個大贏家。 僵局依然難以打破 自2015年後西班牙出現多個新政黨，打破國內傳統左右翼政黨輪替的格局，國會始終無法籌組穩定聯合政府，觸發一次又一次的大選。2019年4月西班牙剛大選結束，因工人社會黨無法取得過半數議席，亦無法拉攏其他政黨支持，籌組聯合政府失敗，導致要一年內二度舉行大選。 極右呼聲黨得助於加泰羅尼亞危機，取得更多右翼選民的支持。（路透社） 觀看目前狀況，西班牙政治局勢仍然沒有改善跡象，工人社會黨仍需要與其他政黨尋求組成聯合政府。儘管極左政黨「我們能」黨（United We Can）願意提供協助，但兩黨總合起來只有146席，仍缺17席才能取得過半數。 目前工人社會黨需與支持加泰獨立的左翼政黨合作才可籌組聯合政府，但外界認為這選項並不可行。假若籌組政府失敗，西班牙將要再次舉行大選。 加泰危機放大右翼聲音 呼聲黨黨魁阿巴斯卡（Santiago Abascal）表示他們的反移民政策及推動的文化與政治變革獲得認同。阿巴期卡稱他們成功公開討論各種禁忌，並告訴左翼政黨他們並不處於道德高地。 10月中，西班牙最高法院以煽動叛亂罪，判處9名鼓吹加泰獨立人士監禁，激發加泰發生暴力示威。判決並促使西班牙右翼政黨要求桑切斯作出嚴厲的

In [101]:
print_result(4)

input:  美國前總統克林頓（Bill Clinton）與白宮前實習生萊溫斯基（Monica Lewinsky）於1998年爆出性醜聞。時隔23年，這宗陳年戀聞因為萊溫斯基於2021年5月21日的一個Twitter推文再被炒熱。 克林頓與萊溫斯基：圖為2019年5月13日，美國白宮前實習生萊溫斯基在美國紐約市出席活動。（Getty） 克林頓與萊溫斯基：圖為2018年10月18日，美國前總統克林頓在Facebook發布自己的照片。（President Bill Clinton Facebook專頁） 克林頓與萊溫斯基：圖為2021年5月13日，美國前總統克林頓（中）在Twitter發布自己的照片。（Bill Clinton Twitter帳號） 克林頓與萊溫斯基：圖為2018年12月5日，美國白宮前實習生萊溫斯基在美國洛杉磯的活動上發言。（
Getty） 克林頓在任總統期間，與白宮見習生萊溫斯基有一段「不恰當關係」，更差點令他被彈劾下台。（Getty Images） 當年萊溫斯基只有22歲。（Getty Images） 美國前總統克林頓雖逃過被彈劾的命運，然而他與萊溫斯基的醜聞，已在他的人生中留下污點。（路透社） 希拉里可算是最關心政治的第一夫人，丈夫克林頓與實習生萊溫斯基有染後，令希拉里化悲憤為力量，在丈夫卸任後專心在政途上向上游。（Getty Images） 網絡應用程式UberFacts在Twitter上問網民：「你做過最高風險、但最低回報的事情是甚麼？」 👀 https://t.co/lrr5eCeCsA — Monica Lewinsky (she/her) (@MonicaLewinsky) May 21, 2021 萊溫斯基2021年5月21日轉發UberFacts的推文，並以「碌大對眼」的表情符號回答UberFacts的問題。 這個表情符號可代表側目而視、暗示一個人正逃避一個訊息、或需要一段長時間來回應一條令人尷尬的問題。因此，她的回應令人聯想到，她是指與克林頓發生婚外情，是她做過最高風險、但最低回報的事。 有UberFact的「妙問」，又有萊溫斯基的「妙答」，Twitter迅即為此沸騰起來，更一度登上熱門搜尋排行榜。 You win Twitter for the day — The_Sofa_King_Black_Vote (@WalterBa

In [102]:
print_result(5)

input:  日前網民Sam Ho在「新手假日行山群組」發帖，指2月7日行經河背水塘時，見到有人違例放生。他向記者表示，當天下午約2時半經過該處，見到一名男子駕車將一個個發泡膠箱運到現場，從箱中取出一袋袋生物，並將牠們倒入水塘，當中包括了水魚、黃鱔和田雞。從相中可見，大量的田雞落入水塘後均見「反肚」，似已奄奄一息。

有途人經過與放生男子爭執
根據水務署資料，在水塘「放生」違反《水務設施條例》第30條，一經循簡易程序定罪，可處第五級罰款（最高為港幣五萬元）及監禁兩年。Sam指，當時有途人經過與該名男子爭執，亦有職員從漁護署車輛走下了解事件，據知職員曾致電通知上司，但半小時後仍沒有行動，最後不了了之；而該名男子曾自稱是甲龍村原居民，一年均會到上址幾次進行放生。而事件引起一眾山友擔心，動物死屍會否帶來其他疫症。

生態教育及資源中心科學事務經理張瑪珊指，胡亂放生對生態方面有很大影響。她指出，雖然田雞、水魚、黃鱔都是淡水生物，但牠們多由市場購入，在運輸的過程中很可能會受傷或者生病，基本上不適合於水塘放生，而且平日食用的田雞大部分不是原生的虎皮蛙，假如將之放生，有機會在本地生態中引入外來物種。


張補充指，「首先如果佢哋唔適應本地生態，可能會即刻就死；如果冇即刻死，喺水塘中佢哋可能同本地物種發生競爭，影響水塘嘅生態平衡。」張瑪珊表示，如果動物不能適應水塘環境而死去，屍體會腐爛影響水質，亦分解成養分，刺激藻類生長甚至爆發，之後影響水塘的含氧量並殺死其他水中生物。此外，如動物大量死去，牠們有機會在分解過程之中滋生細菌，讓生活中於水塘中的魚類染病；而一些本來已經染病的動物，將牠們「放生」便有機會將不同嘅病菌帶入水塘中，使當中的動物染病。
張認為，長遠而言，政府應該立法規管放生行為，需有效地監控用以放生的物種，從而防止不適當的外來物種被人放生，以及有人因為放生而濫捕瀕危的物種。


漁護署人員回覆指，昨日（2月8日）駕駛工作車輛途經清潭水塘前往進行清潔工作時，有市民指有人在水塘進行放生活動。漁護署已即時聯絡相關郊野公園職員到場處理，並留在現場等候，期間曾嘗試勸止但不成功，涉事人約半小時後自行離去。該署郊野公園職員到場，並與在場人士了解事件經過，指會繼續留意該處的情況，亦會就有人在水務設施進行放生活動，轉介相關部門跟進。
漁護署一向不鼓勵市民於野外放生動物，並透過各種渠道加強教育

In [111]:
print_result(10)

input:  英國外交大臣藍韜文（Dominic Raab）香港時間3月17日晚上出席美國阿斯彭安全論壇的虛擬論壇時稱，英國在地球、全球經濟，生態系統有道德責任以及不可分割的利益。惟英媒前一天（16日）報道被人秘密錄音的內容，他稱英國想與違反人權問題的國家做生意。他說因人權問題而限制英國的貿易，意味國家會錯過「增長市場」。此舉惹來部分政界人士批評。 《赫芬頓郵報》16日報道藍韜文的視像會議錄音，當時他面向外交、國協及發展事務部（Foreign, Commonwealth and Development Office）職員，在一個問答環節發言。他稱堅信英國應在全球範圍自由貿易。他說如英國限制自己僅與符合歐洲人權公約的國家做生意，國家未來將不會與「增長市場」有很多貿易協議。 他稱：「我們不會因為（與特定國家）有問題而破壞彼此關係——我們對話因我們想改變（它們的）行為。」（“We don’t junk whole relationships because we’ve got issues – we have a conversation because we want to change the behaviour."） 藍韜文的錄音，被批與他對外表達英國捍衛人權的說法大相庭徑。 約翰遜拒絕採用中國政府在新疆地區犯下「種族滅絕和反人類罪」這一說法，主導英國下議院否決相關提案。圖為2020年12月19日，英國首相約翰遜召開記者會。（Reuters） 《赫芬頓郵報》稱他的這番言論之前，英國政府16日公布《置身競爭時代的全球化英國：安全、國防、發展與外交政策綜合審查》（Global Britain in a Competitive Age: the Integrated Review of Security, Defence, Development and Foreign Policy）這份政策文件。有英國政界人士稱，內文顯示英國低調處理中國的問題。 藍韜文在阿斯彭安全論壇的言論，似是回應有關批評。有記者問他，為何英國推進與中國建立更緊密聯繫，這與美國拜登政府取態有異。他說：「我們是這樣看事情：中國是已然存在（China is here to stay）。」 他說：「我不相信我們會重提冷戰思維或範式。」他稱：「我們是正面的，以及我們應（與中國）尋求在一些範疇有建設性的互動

In [104]:
print_result(8)

input:  台觀光局指出，先前已宣布2月1日至29日暫停旅行業組團前往中國大陸地區旅遊（含轉機前往其他地區旅遊）。為配合台疾病管制署提升港澳地區旅遊疫情等級為第二級，故2月6日起暫停台灣旅行社組團到港澳旅遊，但不含中轉港澳轉機到其他地區。台觀光局表示，持續配合台疾管署相關防疫作為，並視台疫情指揮中心發佈的疫情訊息綜合評估，隨時調整相關管制事宜。一、台籍人士：2月6日起，有陸港澳旅遊史者，需居家檢疫14天；申請獲准至港澳入境者，需自主健康管理14天。二、大陸人士：暫緩入境。三、港澳人士：2月7日起，入境後需居家檢疫14天。四、外籍人士：2月7日起，14天內曾經入境或居住於中國大陸、香港、澳門的外籍人士，暫緩入境。
prediction result:  [SEP] [CLS] 因 應 新 型 冠 狀 病 毒 肺 炎 （ 俗 稱 「 武 漢 肺 炎 」 ） 疫 情 ， 台 灣 觀 光 局 今 天 （ 7 日 ） 宣 布 ， 暫 停 台 灣 旅 行 社 組 團 到 港 澳 旅 遊 ， 但 不 含 中 轉 港 澳 轉 機 [SEP]
label:  考量武漢肺炎疫情持續擴散，台灣觀光局在2月6日宣布，即日起至2月29日，暫停台灣旅行社組團前往香港、澳門地區旅遊，但不包括從港澳轉機。


In [105]:
print_result(9)

input:  Apple Best of 2019 的 Apps 趨勢是用家說故事，而 Unfold 更是其中一個得獎應用程式。 Unfold 與我們平日用的執相 App不同，它並沒有美肌長腿功能，反而提供超過150種相框(當然有些需要課金～)，還有有動感的 GIFs，令到成品更多元化。

不要以為免費相框便沒有好東西，免費相框大致可分為3類：一張圖加字、兩張圖及短故事，大家可以按需要選擇，然後再為相片更改大小，甚至加入濾鏡，張原圖的風格改變，例如 Reykir 將相片的啡黃色突出，會帶點秋意的感覺。處理相片後，便可以插入文字、貼圖和 GIFs，即使沒有課金，選擇亦有很多。不過有個小Tips給大家，在 Unfold 中是不能打中文字，但只要你在 App以外預先打好，然後Copy and Paste 便顯示句子。

執圖後，可以選擇把成品儲存到手機，又或是直接上載到社交媒體，Facebook、Instagram 和 Snapchat，非常方便，除此以外，作為 App 版相簿的 Unfold 可以建立相簿，隨時可以重溫或修改相片。大家可以小上牛刀，先用免費版本的 Unfold，想進一步砌靚圖，再課金每年 HKD158 (或月費 HKD23)！
prediction result:  [SEP] [CLS] 【 apple best of 2019 】 今 年 度 apps 趨 勢 ， 大 家 可 以 在 app 上 執 相 app ， 又 或 是 用 家 說 故 事 的 應 用 程 式 。 unfold 不 但 提 供 美 肌 長 腿 功 能 ， 反 而 提 供 超 過 150 種 [SEP]
label:  今時今日除了 post 相需要執圖，post Story 也不可馬虎了事，假如你不需要每張相也美圖變瘦，以下這個App - Unfold ，可說是來年必備之選，2分鐘執相更可直接上載到社交平台，就讓呀脆跟大家分享一下！
